In [ ]:
# Import required libraries
import os
import cv2
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow/Keras imports for deep learning
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Scikit-learn imports for data splitting and evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

: 

In [ ]:
# Custom preprocessing module for palm images
DATASET_PATH = "dataset"
from img_preprocessing import PalmPreprocessor

# ========== DATA LOADING AND PREPROCESSING ==========
# Initialize lists to store images and corresponding labels
images, labels = [], []
# Get sorted list of class names (person names) from dataset directory
class_names = sorted(os.listdir(DATASET_PATH))

# Iterate through each class (person) in the dataset
for idx, cls in enumerate(class_names):
    cls_path = os.path.join(DATASET_PATH, cls)  # Path to current class folder

    # Process each image in the current class folder
    for img_name in os.listdir(cls_path):
        img_path = os.path.join(cls_path, img_name)  # Full path to image
        
        # Use custom preprocessor to preprocess the palm image
        processed = PalmPreprocessor().preprocess_v2(img_path)

        # Skip if preprocessing fails
        if processed is None:
            import time
            time.sleep(5)  # Wait before trying next image
            continue

        # Add processed image and label to lists
        images.append(processed)
        labels.append(idx)  # Use index as numerical label

# Convert lists to numpy arrays for model training
images = np.array(images)
# Convert labels to one-hot encoded format (e.g., [0,0,1,0] for class 2)
labels = to_categorical(labels, num_classes=len(class_names))

In [ ]:
# ========== DATA SPLITTING ==========
# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, shuffle=True
)

In [ ]:
# Define image size (model expects 128x128 grayscale images)
IMG_SIZE = 128


# ========== CNN MODEL ARCHITECTURE ==========
# Create Sequential model (linear stack of layers)
model = Sequential([
    # First convolutional layer: 32 filters, 3x3 kernel, ReLU activation
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    # Max pooling to reduce spatial dimensions by half
    MaxPooling2D(2,2),

    # Second convolutional layer: 64 filters
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Third convolutional layer: 128 filters
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Flatten 3D feature maps to 1D vector for dense layers
    Flatten(),
    # Fully connected layer with 128 neurons
    Dense(128, activation='relu'),
    # Dropout layer to prevent overfitting (50% neurons randomly disabled)
    Dropout(0.5),
    # Output layer with softmax activation for multi-class classification
    Dense(len(class_names), activation='softmax')
])

In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Low learning rate for stable training
    loss='categorical_crossentropy',       # Standard loss for multi-class classification
    metrics=['accuracy']                   # Track accuracy during training
)

In [ ]:
# Display model architecture summary
model.summary()

In [ ]:
# ========== MODEL TRAINING ==========
# Train the model for 20 epochs with 16 samples per batch
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2  # Use 20% of training data for validation
)

In [ ]:
# ========== TRAINING VISUALIZATION ==========
# Create figure with 2 subplots for training metrics
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: Training and Validation Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Accuracy")
axes[0].set_title("Model Accuracy")
axes[0].legend()
axes[0].grid(True, alpha=0.3)  # Add subtle grid for better readability

# Plot 2: Training and Validation Loss
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Loss")
axes[1].set_title("Model Loss")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
# ========== MODEL EVALUATION ==========
# Evaluate model performance on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

# Generate predictions for test set
y_pred = np.argmax(model.predict(X_test), axis=1)  # Get class index with highest probability
y_true = np.argmax(y_test, axis=1)  # Convert one-hot back to class indices

# Display confusion matrix and classification report
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# ========== SAVE TRAINED MODEL ==========
# Save the complete trained model to disk for later use
model.save("palm_recognition_model.h5")
print("✓ Model saved successfully")

In [ ]:
# ========== CREATE EMBEDDING MODEL ==========
# Load the saved model fresh to ensure proper state
model_for_prediction = load_model("palm_recognition_model.h5")

# Create a new model that outputs embeddings (features from the second-to-last layer)
# This is useful for similarity comparisons and face/palm recognition systems
embedding_model = Model(
    inputs=model_for_prediction.inputs,           # Same input as original model
    outputs=model_for_prediction.layers[-2].output  # Output from Dense(128) layer
)

# ========== GENERATE REFERENCE EMBEDDINGS ==========
# Create a dictionary to store average embeddings for each person
reference_embeddings = {}
EMBEDDING_THRESHOLD = 1.2  # Threshold for similarity matching (not used here but defined for future)

# Generate embeddings for each person in the dataset
for person in class_names:
    embeddings = []
    person_path = os.path.join(DATASET_PATH, person)
    
    # Process all images for the current person
    for img_name in os.listdir(person_path):
        # Skip non-image files
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
            
        img_path = os.path.join(person_path, img_name)
        
        # Use same preprocessing as during training
        preprocessor = PalmPreprocessor()
        processed = preprocessor.preprocess_v2(img_path)
        
        if processed is None:
            continue
            
        # Reshape to match model input shape: (1, 128, 128, 1)
        if len(processed.shape) == 2:
            processed = processed.reshape(1, IMG_SIZE, IMG_SIZE, 1)
        else:
            processed = processed.reshape(1, IMG_SIZE, IMG_SIZE, 1)
        
        # Extract embedding (128-dimensional feature vector)
        emb = embedding_model.predict(processed, verbose=0)[0]
        embeddings.append(emb)
    
    # Store average embedding if valid images were found
    if embeddings:
        reference_embeddings[person] = np.mean(embeddings, axis=0)
        print(f"✓ Generated embeddings for {person}: {len(embeddings)} samples")
    else:
        print(f"✗ No valid images found for {person}")

# Save embeddings to disk for later use in recognition
with open("reference_embeddings.pkl", "wb") as f:
    pickle.dump(reference_embeddings, f)
print("✓ Embeddings saved successfully")

In [ ]:
# ========== TEST PREDICTION WITH RANDOM SAMPLE ==========
# Threshold for classifying as "Unknown" (if confidence is too low)
CONFIDENCE_THRESHOLD = 0.6

# Randomly select a class for testing
true_label = random.choice(class_names)
class_path = os.path.join(DATASET_PATH, true_label)

# Get list of valid image files in the selected class
valid_images = [f for f in os.listdir(class_path) 
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not valid_images:
    print(f"No valid images found in {class_path}")
else:
    # Randomly select an image from the class
    img_name = random.choice(valid_images)
    img_path = os.path.join(class_path, img_name)
    
    # Preprocess the test image using the same pipeline
    preprocessor = PalmPreprocessor()
    processed = preprocessor.preprocess_v2(img_path)
    
    if processed is not None:
        # Reshape for model input
        if len(processed.shape) == 2:
            img_input = processed.reshape(1, IMG_SIZE, IMG_SIZE, 1)
        else:
            img_input = processed.reshape(1, IMG_SIZE, IMG_SIZE, 1)
        
        # Get prediction probabilities for all classes
        predictions = model_for_prediction.predict(img_input, verbose=0)[0]
        confidence = np.max(predictions)  # Highest probability
        predicted_index = np.argmax(predictions)  # Index of predicted class
        
        # Apply unknown detection logic
        if confidence < CONFIDENCE_THRESHOLD:
            predicted_label = "Unknown"  # Classify as unknown if confidence is low
        else:
            predicted_label = class_names[predicted_index]  # Otherwise use predicted class
        
        # Load original image for visualization
        img = cv2.imread(img_path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Create visualization with original and processed images
        plt.figure(figsize=(8, 4))
        
        # Plot original image
        plt.subplot(1, 2, 1)
        plt.imshow(img_gray, cmap='gray')
        plt.title(f"Original\nClass: {true_label}")
        plt.axis('off')
        
        # Plot processed image with prediction results
        plt.subplot(1, 2, 2)
        plt.imshow(processed.squeeze(), cmap='gray')
        plt.title(f"Processed\nPred: {predicted_label}\nConf: {confidence:.2f}")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed prediction results
        print(f"\nPrediction Results:")
        print(f"  Original class: {true_label}")
        print(f"  Predicted: {predicted_label}")
        print(f"  Confidence: {confidence:.2%}")
        print(f"  Correct: {true_label == predicted_label}")
        
        # Display probabilities for all classes
        print("\nAll class probabilities:")
        for i, cls in enumerate(class_names):
            print(f"  {cls}: {predictions[i]:.2%}")
    else:
        print(f"Failed to preprocess image: {img_path}")